In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_csv('yeast.ssv', sep='\s+')
raw_data.head

<bound method NDFrame.head of       ADT1_YEAST  0.58  0.61  0.47  0.13  0.50  0.00  0.48  0.22  MIT
0     ADT2_YEAST  0.43  0.67  0.48  0.27   0.5   0.0  0.53  0.22  MIT
1     ADT3_YEAST  0.64  0.62  0.49  0.15   0.5   0.0  0.53  0.22  MIT
2     AAR2_YEAST  0.58  0.44  0.57  0.13   0.5   0.0  0.54  0.22  NUC
3     AATM_YEAST  0.42  0.44  0.48  0.54   0.5   0.0  0.48  0.22  MIT
4     AATC_YEAST  0.51  0.40  0.56  0.17   0.5   0.5  0.49  0.22  CYT
...          ...   ...   ...   ...   ...   ...   ...   ...   ...  ...
1478  YUR1_YEAST  0.81  0.62  0.43  0.17   0.5   0.0  0.53  0.22  ME2
1479  ZIP1_YEAST  0.47  0.43  0.61  0.40   0.5   0.0  0.48  0.47  NUC
1480  ZNRP_YEAST  0.67  0.57  0.36  0.19   0.5   0.0  0.56  0.22  ME2
1481  ZUO1_YEAST  0.43  0.40  0.60  0.16   0.5   0.0  0.53  0.39  NUC
1482  G6PD_YEAST  0.65  0.54  0.54  0.13   0.5   0.0  0.53  0.22  CYT

[1483 rows x 10 columns]>

In [3]:
X = raw_data.iloc[:, 1:-1].to_numpy()
X

array([[0.43, 0.67, 0.48, ..., 0.  , 0.53, 0.22],
       [0.64, 0.62, 0.49, ..., 0.  , 0.53, 0.22],
       [0.58, 0.44, 0.57, ..., 0.  , 0.54, 0.22],
       ...,
       [0.67, 0.57, 0.36, ..., 0.  , 0.56, 0.22],
       [0.43, 0.4 , 0.6 , ..., 0.  , 0.53, 0.39],
       [0.65, 0.54, 0.54, ..., 0.  , 0.53, 0.22]])

In [4]:
Y = raw_data.iloc[:, -1].to_numpy()
Y

array(['MIT', 'MIT', 'NUC', ..., 'ME2', 'NUC', 'CYT'], dtype=object)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
skf = StratifiedKFold(n_splits=5)
scaler = StandardScaler()
iteration=0
for train_index, test_index in skf.split(X,Y):
        iteration = iteration+1
        X_train, X_test, Y_train, Y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]
        X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.2) #actually splitting validation, not test
        
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform (X_test)
        X_validation = scaler.transform (X_validation)
        
        max_acc=0
        max_k=-1
        for k in range(2, 16):
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, Y_train)
            acc = knn.score(X_validation, Y_validation)
            if (acc>max_acc):
                max_acc=acc
                max_k=k
        print("melhor k:",max_k)
                
        X_train = np.concatenate((X_train, X_validation))
        Y_train = np.concatenate((Y_train, Y_validation))
        knn = KNeighborsClassifier(n_neighbors=max_k)
        knn.fit(X_train, Y_train)
        print("Score do knn:", knn.score(X_test, Y_test), "")

melhor k: 13
Score do knn: 0.6868686868686869 
melhor k: 15
Score do knn: 0.632996632996633 
melhor k: 13
Score do knn: 0.6666666666666666 
melhor k: 7
Score do knn: 0.6655405405405406 
melhor k: 15
Score do knn: 0.625 
